# 載入使用套件

In [ ]:
!pip install tomorrow

  Created wheel for tomorrow: filename=tomorrow-0.2.4-cp36-none-any.whl size=3130 sha256=bd6844cab4d39b827b405ae21e0ae03efbff31a774e383dec5482af76060d215
  Stored in directory: /root/.cache/pip/wheels/b0/47/cc/73e5cdc873528fd7933dc4d8cdeb67b899fe9c263d0cbc037f
Successfully built tomorrow


In [ ]:
# 爬蟲類套件
import requests
import json
from bs4 import BeautifulSoup
# 加入使用者資訊(如使用什麼瀏覽器、作業系統...等資訊)模擬真實瀏覽網頁的情況
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
from tomorrow import threads
# 資料整理類套件
import pandas as pd
import re
from datetime import datetime

# 其他
import os

# 擷取一篇新聞

In [ ]:
def GetNews_chinatimes(response):
    soup = BeautifulSoup(response.text)
    url = soup.find('link')['href']
    ndf = pd.DataFrame(data = [{'TITLE':soup.find('h1', attrs={'class':'article-title'}).text,
                                'TIME':datetime.strptime(soup.find('meta', attrs={'property':'article:published_time'})['content'],'%Y-%m-%dT%H:%M:%S+08:00'),
                                'CATEGORY':soup.find('meta',attrs={'property':'article:section'})['content'],
                                'DESCRIPTION':soup.find('meta',attrs={'name':'description'})['content'],
                                'CONTENT':'\n'.join(i.text for i in soup.find('div',attrs={'class':'article-body'}).find_all('p')),
                                'KEYWORDS':soup.find('meta',{'name':'news_keywords'})['content'],
                                'FROM':soup.find('meta',{'name':'publisher'})['content'],
                                'LINK':soup.find('meta', {'property':'og:url'})['content']}],
                       columns = ['TITLE', 'TIME', 'CATEGORY', 'DESCRIPTION', 'CONTENT','KEYWORDS', 'FROM', 'LINK']) 
    return ndf

In [ ]:
url = 'https://www.chinatimes.com/newspapers/20191028000424-260118?chdtv'
resp = requests.get(url)
GetNews_chinatimes(resp)

,TITLE,TIME,CATEGORY,DESCRIPTION,CONTENT,KEYWORDS,FROM,LINK
0,直球對決 韓接招青年辣提問！可以討厭我 但請聽我的政策,2019-10-28 04:10:19,政治要聞,國民黨總統參選人韓國瑜昨晚在台北舉辦青年論壇，標榜「走出同溫層」，果然在場青年提問辛辣，毫不...,國民黨總統參選人韓國瑜昨晚在台北舉辦青年論壇，標榜「走出同溫層」，果然在場青年提問辛辣，毫不...,"韓國瑜,國民黨,2020大選",中時電子報,https://www.chinatimes.com/newspapers/20191028...


# 擷取特定關鍵詞新聞的連結清單

In [ ]:
def GetLinks_chinatimes(response):
    links = []
    soup = BeautifulSoup(response.text)
    for i in soup.find_all('h3'):
        url = i.find('a')['href']
        links.append(url)
    return links

In [ ]:
url = 'https://www.chinatimes.com/search/美國'
resp = requests.get(url)
GetLinks_chinatimes(resp)

['https://www.chinatimes.com/newspapers/20200131000218-260203',
 'https://www.chinatimes.com/realtimenews/20200131001427-260408',
 'https://www.chinatimes.com/realtimenews/20200131000791-260408',
 'https://www.chinatimes.com/newspapers/20200131000370-260208',
 'https://www.chinatimes.com/newspapers/20200131000217-260203',
 'https://www.chinatimes.com/newspapers/20200131000375-260208',
 'https://www.chinatimes.com/newspapers/20200131000200-260203',
 'https://www.chinatimes.com/newspapers/20200131000117-260301',
 'https://www.chinatimes.com/realtimenews/20200131003148-260410',
 'https://www.chinatimes.com/realtimenews/20200131002544-260410',
 'https://www.chinatimes.com/realtimenews/20200131001828-260407',
 'https://www.chinatimes.com/hottopic/20200131003838-260804',
 'https://www.chinatimes.com/opinion/20200131000004-262115',
 'https://www.chinatimes.com/realtimenews/20200131000771-260408',
 'https://www.chinatimes.com/realtimenews/20200131001564-260408',
 'https://www.chinatimes.com/ho

## 開啟多線程功能

In [ ]:
@threads(5)
def MultiThread_Crawl(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass

# 組合應用

In [ ]:
def CrawlNews_chinatimes(keywords, pages):
    # 截取多個分頁的新聞連結
    links = []
    for i in range(pages):
        url = 'https://www.chinatimes.com/search/{}?page={}'.format(keywords, i+1)
        resp = requests.get(url)
        links += GetLinks_chinatimes(resp)
        links = list(set(links))  
        print('There are {} links in page {}.'.format(len(links),str(i)))

    # 多線程爬蟲
    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    list_of_dataframes = []
    for response in responses:
        try:
            ndf = GetNews_chinatimes(response)
            list_of_dataframes.append(ndf)
        except:
            pass
    df = pd.concat(list_of_dataframes, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    return df

- 爬蟲前可以先手動查詢大約有多少新聞

In [ ]:
%%time
# 可以自行替換查詢的關鍵字，另外需要更多新聞的人也可以把 pages 的數值調高)
df = CrawlNews_chinatimes(keywords='台積電', pages=10)

There are 20 links in page 0.
There are 40 links in page 1.
There are 60 links in page 2.
There are 80 links in page 3.
There are 100 links in page 4.
There are 120 links in page 5.
There are 136 links in page 6.
There are 156 links in page 7.
There are 176 links in page 8.
There are 196 links in page 9.
There are 195 News in DataFrame.
CPU times: user 8.9 s, sys: 292 ms, total: 9.19 s
Wall time: 23.4 s


In [ ]:
df

,TITLE,TIME,CATEGORY,DESCRIPTION,CONTENT,KEYWORDS,FROM,LINK
0,《盤後解析》月線有撐守萬二，封關前提防賣壓,2020-01-16 14:32:06,財經,受到台積電(2330)下挫5.5元，影響指數達50點，台股開低走低，終場以12066.93點...,受到台積電(2330)下挫5.5元，影響指數達50點，台股開低走低，終場以12066.93點...,"台積電,指數",中時電子報,https://www.chinatimes.com/realtimenews/202001...
1,台積電前瞻佈局賽 元智大學奪首獎,2020-01-21 04:10:06,產業．科技,晶圓代工龍頭台積電20日宣布，首屆「前瞻佈局大賽」經過為期4個月的賽程，兩階段的競賽，由元智...,晶圓代工龍頭台積電20日宣布，首屆「前瞻佈局大賽」經過為期4個月的賽程，兩階段的競賽，由元智...,台積電,中時電子報,https://www.chinatimes.com/newspapers/20200121...
2,嚇！台積電ADR慘崩 台股後天難逃暴跌,2020-01-28 09:26:06,財經,武漢肺炎疫情持續延燒，市場擔憂恐影響全球經濟成長，華爾街投資人驚嚇狂賣股票，美股三大指數全面...,武漢肺炎疫情持續延燒，市場擔憂恐影響全球經濟成長，華爾街投資人驚嚇狂賣股票，美股三大指數全面...,"半導體,台積電,道瓊,美股,肺炎,疫情,美光,英特爾,輝達",中時電子報,https://www.chinatimes.com/realtimenews/202001...
3,12檔外資狂敲 扮明燈,2020-01-31 04:10:01,證券．權證,台股30日重挫近700點，但外資只賣超近181億元，投資專家指出，外資換手激烈，不要只看賣超...,台股30日重挫近700點，但外資只賣超近181億元，投資專家指出，外資換手激烈，不要只看賣超...,台股,中時電子報,https://www.chinatimes.com/newspapers/20200131...
4,聯發科5G晶片夯 陸手機廠預付訂金,2020-01-30 04:10:00,產業．科技,5G智慧手機已經在2020年如雨後春筍般問世，品牌廠更排定全年的新機發表計畫，5G手機晶片已...,5G智慧手機已經在2020年如雨後春筍般問世，品牌廠更排定全年的新機發表計畫，5G手機晶片已...,"台積電,產能",中時電子報,https://www.chinatimes.com/newspapers/20200130...
...,...,...,...,...,...,...,...,...
190,老美斷陸企肥單慘了？ 台積電7奈米大功臣沒在怕,2020-01-27 14:30:00,財經,半導體設備大廠荷蘭商艾司摩爾（ASML）獨家供應的EUV光刻設備，為全球晶圓代工龍頭台積電、...,半導體設備大廠荷蘭商艾司摩爾（ASML）獨家供應的EUV光刻設備，為全球晶圓代工龍頭台積電、...,"ASML,台積電, EUV,中芯國際",中時電子報,https://www.chinatimes.com/realtimenews/202001...
191,逐洞賽》倫元投顧分析師陳學進,2020-01-20 17:10:57,財經,倫元投顧分析師陳學進表示，台股20日開高震盪收紅，終場上漲28.42點，漲幅0.24％，收在...,倫元投顧分析師陳學進表示，台股20日開高震盪收紅，終場上漲28.42點，漲幅0.24％，收在...,"台積電,倫元投顧分析師陳學進,台股,終場,漲幅",中時電子報,https://www.chinatimes.com/realtimenews/202001...
192,《晨間解盤》短線震盪，擇優布局(日盛投顧提供),2020-01-20 09:16:25,財經,美國上周五(1/17)新屋開工數據的亮麗，使得投資人樂觀情緒延續美股再創新高，終場美股四大指...,美國上周五(1/17)新屋開工數據的亮麗，使得投資人樂觀情緒延續美股再創新高，終場美股四大指...,"台股,解盤",中時電子報,https://www.chinatimes.com/realtimenews/202001...
193,全押台積電太可惜！ 封關前看這幾檔黑馬奔騰,2020-01-19 17:40:00,財經,台股在二○一九年漲勢驚人，加權指數從年初的九三一九點起漲，到十二月三十一日最後交易日以一一九...,台股在二○一九年漲勢驚人，加權指數從年初的九三一九點起漲，到十二月三十一日最後交易日以一一九...,"記憶體,台股,台積電,聯發科",中時電子報,https://www.chinatimes.com/realtimenews/202001...


# 儲存結果

## 串接 Google Drive 空間

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## 保存資料
- 資料會保存在 Google Drive 上 Colab Notebooks/WebCrawler/TW_News/News 的資料夾空間中

In [ ]:
import os
path = '/content/drive/My Drive/Colab Notebooks/WebCrawler/TW_News/News'
# 如果不存在這個資料夾就新建這個資料夾
if not os.path.exists(path):
    os.makedirs(path)
df.to_excel(path + '/News_chinatimes.xlsx')